<a href="https://colab.research.google.com/github/3HDevelopment/DataScience/blob/main/Final_Assignment_Databases_and_SQL_for_Data_Science_with_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%load_ext sql

In [2]:
import csv, sqlite3

con = sqlite3.connect("RealWorldData.db")
cur = con.cursor()

In [3]:
!pip install -q pandas==1.1.5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 12.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... canceled
ERROR: Could not install packages due to an OSError: [Errno 9] Bad file descriptor



In [4]:
%load_ext sql
%sql sqlite:///RealWorldData.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [5]:
import pandas
df = pandas.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCensusData.csv")
df.to_sql("CENSUS_DATA", con, if_exists='replace', index=False,method="multi")

df = pandas.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCrimeData.csv")
df.to_sql("CHICAGO_CRIME_DATA", con, if_exists='replace', index=False, method="multi")

df = pandas.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoPublicSchools.csv")
df.to_sql("CHICAGO_PUBLIC_SCHOOLS_DATA", con, if_exists='replace', index=False, method="multi")

566

In [ ]:
%sql select * from CHICAGO_PUBLIC_SCHOOLS_DATA LIMIT 10;

In [ ]:
%sql select * from CHICAGO_CRIME_DATA LIMIT 10;

In [ ]:
%sql select * from CENSUS_DATA limit 1;

**Problem 1**
Find the total number of crimes recorded in the CRIME table.

In [6]:
%sql select count(ID) from CHICAGO_CRIME_DATA;

 * sqlite:///RealWorldData.db
Done.


count(ID)
533


**Problem 2**
List community areas with per capita income less than 11000.¶

In [34]:
%sql SELECT COMMUNITY_AREA_NAME, PER_CAPITA_INCOME FROM CENSUS_DATA WHERE PER_CAPITA_INCOME < 11000;

 * sqlite:///RealWorldData.db
Done.


COMMUNITY_AREA_NAME,PER_CAPITA_INCOME
West Garfield Park,10934
South Lawndale,10402
Fuller Park,10432
Riverdale,8201


**Problem 3**
List all case numbers for crimes involving minors?(children are not considered minors for the purposes of crime analysis)

In [36]:
%sql SELECT DISTINCT CASE_NUMBER, DESCRIPTION FROM CHICAGO_CRIME_DATA WHERE DESCRIPTION LIKE '%MINOR%';

 * sqlite:///RealWorldData.db
Done.


CASE_NUMBER,DESCRIPTION
HL266884,SELL/GIVE/DEL LIQUOR TO MINOR
HK238408,ILLEGAL CONSUMPTION BY MINOR


**Problem 4**
List all kidnapping crimes involving a child?

In [37]:
%sql SELECT DISTINCT CASE_NUMBER, PRIMARY_TYPE, DATE, DESCRIPTION FROM CHICAGO_CRIME_DATA \
WHERE PRIMARY_TYPE='KIDNAPPING'

 * sqlite:///RealWorldData.db
Done.


CASE_NUMBER,PRIMARY_TYPE,DATE,DESCRIPTION
HN144152,KIDNAPPING,2007-01-26,CHILD ABDUCTION/STRANGER


**Problem 5**
What kinds of crimes were recorded at schools?

In [41]:
%sql SELECT DISTINCT(PRIMARY_TYPE), LOCATION_DESCRIPTION FROM CHICAGO_CRIME_DATA WHERE LOCATION_DESCRIPTION LIKE '%SCHOOL%';

 * sqlite:///RealWorldData.db
Done.


PRIMARY_TYPE,LOCATION_DESCRIPTION
BATTERY,"SCHOOL, PUBLIC, GROUNDS"
BATTERY,"SCHOOL, PUBLIC, BUILDING"
CRIMINAL DAMAGE,"SCHOOL, PUBLIC, GROUNDS"
NARCOTICS,"SCHOOL, PUBLIC, GROUNDS"
NARCOTICS,"SCHOOL, PUBLIC, BUILDING"
ASSAULT,"SCHOOL, PUBLIC, GROUNDS"
CRIMINAL TRESPASS,"SCHOOL, PUBLIC, GROUNDS"
PUBLIC PEACE VIOLATION,"SCHOOL, PRIVATE, BUILDING"
PUBLIC PEACE VIOLATION,"SCHOOL, PUBLIC, BUILDING"


**Problem 6**
List the average safety score for each type of school.

In [64]:
%sql SELECT "Elementary, Middle, or High School" AS School_Type, AVG(SAFETY_SCORE) FROM CHICAGO_PUBLIC_SCHOOLS_DATA \
GROUP BY "Elementary, Middle, or High School";

 * sqlite:///RealWorldData.db
Done.


School_Type,AVG(SAFETY_SCORE)
ES,49.52038369304557
HS,49.62352941176471
MS,48.0


**Problem 7**
List 5 community areas with highest % of households below poverty line

In [67]:
%sql SELECT COMMUNITY_AREA_NAME, PERCENT_HOUSEHOLDS_BELOW_POVERTY from CENSUS_DATA\
ORDER BY PERCENT_HOUSEHOLDS_BELOW_POVERTY DESC LIMIT 5;

 * sqlite:///RealWorldData.db
Done.


COMMUNITY_AREA_NAME,PERCENT_HOUSEHOLDS_BELOW_POVERTY
Riverdale,56.5
Fuller Park,51.2
Englewood,46.6
North Lawndale,43.1
East Garfield Park,42.4


**Problem 8**
Which community area is most crime prone?

In [79]:
  %sql SELECT COMMUNITY_AREA_NUMBER AS AREA_NUMBER, COUNT(COMMUNITY_AREA_NUMBER) AS CRIME_COUNT FROM CHICAGO_CRIME_DATA\
  GROUP BY COMMUNITY_AREA_NUMBER\
  ORDER BY COUNT(COMMUNITY_AREA_NUMBER) DESC LIMIT 1;

 * sqlite:///RealWorldData.db
Done.


AREA_NUMBER,CRIME_COUNT
25.0,43


**Problem 9**
Use a sub-query to find the name of the community area with highest hardship index

In [85]:
%sql SELECT COMMUNITY_AREA_NAME as AREA_NAME, HARDSHIP_INDEX FROM CENSUS_DATA WHERE HARDSHIP_INDEX = (SELECT MAX(HARDSHIP_INDEX) FROM CENSUS_DATA);

 * sqlite:///RealWorldData.db
Done.


AREA_NAME,HARDSHIP_INDEX
Riverdale,98.0


**Problem 10**
Use a sub-query to determine the Community Area Name with most number of crimes?

In [131]:
%%sql
SELECT CD.community_area_name, MAX(CRD.NUM) AS CRIME_COUNT FROM CENSUS_DATA CD,
(SELECT COUNT(COMMUNITY_AREA_NUMBER) AS NUM FROM CHICAGO_CRIME_DATA GROUP BY COMMUNITY_AREA_NUMBER) AS CRD
WHERE CD.COMMUNITY_AREA_NUMBER = (SELECT COMMUNITY_AREA_NUMBER FROM CHICAGO_CRIME_DATA
    GROUP BY COMMUNITY_AREA_NUMBER
    ORDER BY COUNT(COMMUNITY_AREA_NUMBER) DESC
    LIMIT 1)
LIMIT 1;

 * sqlite:///RealWorldData.db
Done.


COMMUNITY_AREA_NAME,CRIME_COUNT
Austin,43
